<a href="https://colab.research.google.com/github/profsong/wnd_course/blob/main/inugsl_wnd_code1_mip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEST 프로그램입니다.
어떻게 **테스트**하나요?

In [ ]:
!pip install ortools

In [ ]:
from __future__ import print_function
from ortools.linear_solver import pywraplp


def main():
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')

    infinity = solver.infinity()
    # x and y are integer non-negative variables.
    x = solver.IntVar(0.0, infinity, 'x')
    y = solver.IntVar(0.0, infinity, 'y')

    print('Number of variables =', solver.NumVariables())

    # x + 7 * y <= 17.5.
    solver.Add(x + 7 * y <= 17.5)

    # x <= 3.5.
    solver.Add(x <= 3.5)

    print('Number of constraints =', solver.NumConstraints())

    # Maximize x + 10 * y.
    solver.Maximize(x + 10 * y)

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        print('x =', x.solution_value())
        print('y =', y.solution_value())
    else:
        print('The problem does not have an optimal solution.')

    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())


if __name__ == '__main__':
    main()

Number of variables = 2
Number of constraints = 2
Solution:
Objective value = 23.0
x = 3.0
y = 2.0

Advanced usage:
Problem solved in 6.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes


In [ ]:
from ortools.linear_solver import pywraplp

if __name__ == '__main__':



  # Data
  costs = [
      [90, 80, 75, 70],
      [35, 85, 55, 65],
      [125, 95, 90, 95],
      [45, 110, 95, 115],
      [50, 100, 90, 100],
  ]
  num_workers = len(costs)
  num_tasks = len(costs[0])


  #----------------------------------------------------------------
  # Solver :  Create the mip solver with the CBC backend.
  solver = pywraplp.Solver.CreateSolver('CBC')


  #----------------------------------------------------------------
  # Variables
  # x[i, j] is an array of 0-1 variables, which will be 1
  # if worker i is assigned to task j.
  x = {}
  for i in range(num_workers):
      for j in range(num_tasks):
          x[i, j] = solver.IntVar(0, 1, '')


  #----------------------------------------------------------------
  # Constraints
  # Each worker is assigned to at most 1 task.
  for i in range(num_workers):
      solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

  # Each task is assigned to exactly one worker.
  for j in range(num_tasks):
      solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)


  #----------------------------------------------------------------
  # Objective
  objective_terms = []
  for i in range(num_workers):
      for j in range(num_tasks):
          objective_terms.append(costs[i][j] * x[i, j])
  solver.Minimize(solver.Sum(objective_terms))


  #----------------------------------------------------------------
  # Solve
  status = solver.Solve()


  #----------------------------------------------------------------
  # Print solution.
  if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
      print('Total cost = ', solver.Objective().Value(), '\n')
      for i in range(num_workers):
          for j in range(num_tasks):
              # Test if x[i,j] is 1 
              # (with tolerance for floating point arithmetic).
              if x[i, j].solution_value() > 0.5:
                  print('Worker %d assigned to task %d.  Cost = %d' %
                        (i, j, costs[i][j]))


Total cost =  265.0 

Worker 0 assigned to task 3.  Cost = 70
Worker 1 assigned to task 2.  Cost = 55
Worker 2 assigned to task 1.  Cost = 95
Worker 3 assigned to task 0.  Cost = 45
